In [1]:
import pandas as PD
import numpy as NP
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import preprof         #my functions

In [2]:
## LOAD IN DATA

#directory = '/Users/cdw/Desktop/pkpd_script/LABELLED/COMPLETE/'
#preprof.generate_allcsv(directory,'all.csv')
#csv= '/Users/cdw/Desktop/pkpd_script/all.csv'

csv = '/Users/cdw/Desktop/pkpd_script/MNCA_ANALYSIS/ready_processed_2.csv'
papers=PD.read_csv(csv)
relevant = ['Non-compartmental','Modelling']
relevant_papers = papers.loc[papers['category'].isin(relevant)]

x = [a.split('!$!') for a in relevant_papers.loc[:,'words']]
y = relevant_papers.loc[:,'category']

In [3]:
## FEATURE ENGINEERING

def fake_tokeniser(text):
    return text

tf_idf = TfidfVectorizer(analyzer='word',
                         tokenizer=fake_tokeniser,
                         preprocessor=fake_tokeniser,
                         lowercase=False,
                         token_pattern=None)
bag_of_words = CountVectorizer(analyzer='word',
                               tokenizer=fake_tokeniser,
                               preprocessor=fake_tokeniser,
                               lowercase=False,
                               token_pattern=None)

def wm2df(wm, feat_names):  
    doc_names = [f'Doc{idx}' for idx, _ in enumerate(wm)]
    df = PD.DataFrame(data=wm.toarray(), index=doc_names,
                      columns=feat_names)
    return(df)

tfidf = tf_idf.fit_transform(x)
tfidf_names = tf_idf.get_feature_names()

bow=bag_of_words.fit_transform(x)
bow_names=bag_of_words.get_feature_names()

In [4]:
## remove characters '[', ']' and '>' as xgboost doesn't allow them in feature names

def remove_chars(feature_names):
    new_feature_list=[]
    for name in feature_names:
        if '[' in name:
            ind=name.index('[')
            name=name[:ind]+name[ind+1:]
        if ']' in name:
            ind=name.index(']')
            name=name[:ind]+name[ind+1:]
        if '>' in name:
            ind=name.index('>')
            name=name[:ind]+name[ind+1:]
        new_feature_list.append(name)
    return new_feature_list

tfidf_names_fixed=remove_chars(tfidf_names)
bow_names_fixed=remove_chars(bow_names)

In [5]:
## change names of duplicate features

def change_dups(features_fixed):
    counter=0
    for feature in features_fixed:
        counter+=1
        if counter%1000==0:
            print('.',end='')
        if features_fixed.count(feature)>1:
            print('-',end='')
            index_1=features_fixed.index(feature)
            features_fixed[index_1]+='_1'
            index_2=features_fixed.index(feature)
            features_fixed[index_2]+='_2'
            try:
                index_3=features_fixed.index(feature)
                features_fixed[index_3]+='_3'
            except: pass
            try:
                index_4=features_fixed.index(feature)
                features_fixed[index_4]+='_4'
            except: pass
    return features_fixed

tfidf_names_fixed=change_dups(tfidf_names_fixed)
bow_names_fixed=change_dups(bow_names_fixed)

..........................

In [6]:
## drop trickly column
## we  can do this becuase its feature importance is 0

tfidf_matrix = wm2df(tfidf,tfidf_names_fixed)
#tfidf_matrix=tfidf_matrix.drop(columns='(1r,9s,12s,13r,14s,17r,18e,21s,23s,24r,25s,27r)-1,14-dihydroxy-12-(e)-2-(1r,3r,4r)-4-hydroxy-3-methoxycyclohexyl-1-methylvinyl-23,25-dimethoxy-13,19,21,27-tetramethyl-17-(2-oxopropyl)-11,28-dioxa-4-azatricyclo[22.3.1.0(4.9)]octacos-18-ene-2,3,10,16-tetroneT')
bow_matrix=wm2df(bow,bow_names_fixed)
#bow_matrix=bow_matrix.drop(columns='(1r,9s,12s,13r,14s,17r,18e,21s,23s,24r,25s,27r)-1,14-dihydroxy-12-(e)-2-(1r,3r,4r)-4-hydroxy-3-methoxycyclohexyl-1-methylvinyl-23,25-dimethoxy-13,19,21,27-tetramethyl-17-(2-oxopropyl)-11,28-dioxa-4-azatricyclo[22.3.1.0(4.9)]octacos-18-ene-2,3,10,16-tetroneT')

In [7]:
pickle.dump(bow_matrix,open('bow_matrix','wb'))
pickle.dump(bow_matrix,open('tfidf_matrix','wb'))